In [ ]:
!pip install --upgrade pip
!pip install torch 
!pip install accelerate==0.27.2
!pip install peft 
!pip install bitsandbytes 
!pip install transformers 
!pip install trl
!pip install openpyxl
!pip install tensorboardX
!pip install ipywidgets
!pip install tensorboard
!pip install mlflow
!pip install datasets

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Trainer,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_metric
import numpy as np
from sklearn.metrics import accuracy_score

In [3]:
os.environ["DISABLE_MLFLOW_INTEGRATION"] = "TRUE"
# os.environ['MLFLOW_ENABLE_ASYNC_LOGGING'] = 'True'

In [4]:
inputpath = 's3://a208321-ml-workspace-questionnairxyup-use1/FinalDataSet.csv'
Bucket_name='a208321-ml-workspace-questionnairxyup-use1'
filename='FinalDataSet.csv'
path = 's3://{}/{}'.format(Bucket_name,filename)

In [5]:
df = pd.read_csv(path)
df = df.astype(str)
train, test = train_test_split(df, test_size=0.2)

/opt/conda/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [ ]:
# def prepare_dataset(train_df, test_df):
#     # Initialize the tokenizer
#     tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
    
#     # Tokenize the questions and category names for the training dataset
#     train_inputs = tokenizer(train_df['Question'].tolist(), padding=True, truncation=True, return_tensors="pt")
#     train_labels = tokenizer(train_df['TC Category Code'].tolist(), padding=True, truncation=True, return_tensors="pt")
    
#     # Create a Dataset object for training
#     train_dataset = Dataset.from_dict({
#         'input_ids': train_inputs['input_ids'],
#         'attention_mask': train_inputs['attention_mask'],
#         'labels': train_labels['input_ids']
#     })
    
#     # Tokenize the questions and category names for the testing dataset
#     test_inputs = tokenizer(test_df['Question'].tolist(), padding=True, truncation=True, return_tensors="pt")
#     test_labels = tokenizer(test_df['TC Category Code'].tolist(), padding=True, truncation=True, return_tensors="pt")
    
#     # Create a Dataset object for testing
#     test_dataset = Dataset.from_dict({
#         'input_ids': test_inputs['input_ids'],
#         'attention_mask': test_inputs['attention_mask'],
#         'labels': test_labels['input_ids']
#     })
    
#     return train_dataset, test_dataset

# # Assuming train and test are your DataFrames with the training and testing data
# train_dataset, test_dataset = prepare_dataset(train, test)

In [ ]:
import shutil

# Specify the path to the directory you want to clear
directory_path = "./results/checkpoint-500"

# Clear the directory
shutil.rmtree(directory_path)

In [6]:
def prepare_dataset(train_df, test_df):
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
    
    # Concatenate 'Question' and 'SelectedOptionId' to create a new feature for training
    train_df['combined_input'] = train_df['Question'] + ' ' + train_df['SelectedOptionId']
    # Tokenize the combined input and category names for the training dataset
    train_inputs = tokenizer(train_df['combined_input'].tolist(), padding=True, truncation=True, return_tensors="pt")
    train_labels = tokenizer(train_df['TC Category Code'].tolist(), padding=True, truncation=True, return_tensors="pt")
    
    # Create a Dataset object for training
    train_dataset = Dataset.from_dict({
        'input_ids': train_inputs['input_ids'],
        'attention_mask': train_inputs['attention_mask'],
        'labels': train_labels['input_ids']
    })
    
    # Concatenate 'Question' and 'SelectedOptionId' to create a new feature for testing
    test_df['combined_input'] = test_df['Question'] + ' ' + test_df['SelectedOptionId']
    # Tokenize the combined input and category names for the testing dataset
    test_inputs = tokenizer(test_df['combined_input'].tolist(), padding=True, truncation=True, return_tensors="pt")
    test_labels = tokenizer(test_df['TC Category Code'].tolist(), padding=True, truncation=True, return_tensors="pt")
    
    # Create a Dataset object for testing
    test_dataset = Dataset.from_dict({
        'input_ids': test_inputs['input_ids'],
        'attention_mask': test_inputs['attention_mask'],
        'labels': test_labels['input_ids']
    })
    
    return train_dataset, test_dataset

# Assuming train and test are your DataFrames with the training and testing data
train_dataset, test_dataset = prepare_dataset(train, test)

In [7]:
# Define the model
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
)

metric = load_metric('accuracy')

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Debug: Print the shapes of predictions and labels
#     print("Predictions shape:", predictions.shape)
#     print("Labels shape:", labels.shape)
    
#     # Debug: Print the first few predictions and labels
#     print("Predictions:", predictions[:5])
#     print("Labels:", labels[:5])
    
#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
#     compute_metrics=compute_metrics,
)
# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/tmp/ipykernel_2658/1980032826.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this messa

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 14.58 GiB of which 1.62 MiB is free. Process 29141 has 14.34 GiB memory in use. Process 1576 has 226.00 MiB memory in use. Of the allocated memory 126.00 MiB is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Testing for test dataset
test_results = trainer.evaluate(test_dataset)
print(test_results)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")

# Save the model
model.save_pretrained("path/to/save/directory")
tokenizer.save_pretrained("path/to/save/directory")

In [ ]:
# Later, load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("path/to/save/directory")
tokenizer = AutoTokenizer.from_pretrained("path/to/save/directory")

In [ ]:
# Define the list of questions
questions = [
    "You said moved last year. Was this military service related?",
    "You indicated retirement contributions or distributions for this year. Did you or your spouse receive payment from retirement, or severance compensation?",
    "You indicated a job change this year, please upload W2 associated with new job if applicable.",
    "Were you or your spouse the beneficiary of an estate, trust or ABLE account?",
    "Was there any distributions from a 529 College Savings Plan or an Educational Savings Account (need Form 1099-Q)?"
]

# Loop to ask the user each question
for question in questions:
    # Ask the user the current question
    response = input(f"{question} (Yes/No/NA): ")
    
    # Concatenate the question and the user's response
    combined_input = question + " " + response
    
    # Tokenize the combined input
    inputs = tokenizer.encode(combined_input, return_tensors="pt")
    
    # Generate the output sequence
    outputs = model.generate(inputs, max_length=128, min_length=8, top_p=0.9, do_sample=True)
    
    # Decode the output sequence to get the predicted category name
    predicted_category = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Print the predicted category name
    print(f"Predicted Category Name: {predicted_category}")